In [1]:
import os
from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
stim_sig_chans = load_sig_chans(stim_filename)

stim_filename_0to0point5 = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror_window_0to0.5.json'
stim_sig_chans_0to0point5 = load_sig_chans(stim_filename_0to0point5)

stim_filename_0point5to1 = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror_window_0.5to1.json'
stim_sig_chans_0point5to1 = load_sig_chans(stim_filename_0point5to1)

stim_filename_0to1 = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror_window_0to1.json'
stim_sig_chans_0to1 = load_sig_chans(stim_filename_0to1)

stim_sig_chans_all_windows = stim_sig_chans_0to0point5 + stim_sig_chans_0point5to1 + stim_sig_chans_0to1
stim_sig_chans_all_windows = np.unique(stim_sig_chans_all_windows)

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

Loaded significant channels for subject D0103
Loaded significant channels for subject D0103
Loaded significant channels for subject D0103
Loaded significant channels for subject D0103


In [24]:
import os
import json

# Define the subjects list
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0071', 
            'D0077', 'D0090', 'D0100', 'D0102', 'D0103']

# Initialize a dictionary to hold the counts
counts = {
    'total_filt_channels': 0,
    'total_good_channels': 0,
    'total_sig_channels_timepermcluster': 0,
    'total_sig_channels_0to0point5': 0,
    'total_sig_channels_0point5to1': 0,
    'total_sig_channels_0to1': 0,
    'total_sig_channels_all_windows': 0
}

# Define the base path for your files
base_path = 'C:/Users/jz421/Box/CoganLab/BIDS-1.1_GlobalLocal/BIDS/derivatives/'

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Loop through each subject to gather data
for sub in subjects:
    # Load the 'good' channels
    good_channels_path = os.path.join(base_path, 'freqFilt', 'figs', sub, f'channels_{sub}_GlobalLocal.txt')
    with open(good_channels_path, 'r') as file:
        good_channels = file.readlines()  # Adjust this line if the data is stored in a different format (e.g., JSON)
    num_good_channels = len(good_channels)
    
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)

    stim_filename_0to0point5 = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror_window_0to0.5.json'
    stim_sig_chans_0to0point5 = load_sig_chans(stim_filename_0to0point5)

    stim_filename_0point5to1 = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror_window_0.5to1.json'
    stim_sig_chans_0point5to1 = load_sig_chans(stim_filename_0point5to1)

    stim_filename_0to1 = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror_window_0to1.json'
    stim_sig_chans_0to1 = load_sig_chans(stim_filename_0to1)

    stim_sig_chans_all_windows = stim_sig_chans_0to0point5 + stim_sig_chans_0point5to1 + stim_sig_chans_0to1
    stim_sig_chans_all_windows = np.unique(stim_sig_chans_all_windows)

    sig_chans_per_subject['stim_sig_chans_timepermcluster'] = {}
    sig_chans_per_subject['stim_sig_chans_0to0point5'] = {}
    sig_chans_per_subject['stim_sig_chans_0point5to1'] = {}
    sig_chans_per_subject['stim_sig_chans_0to1'] = {}
    sig_chans_per_subject['stim_sig_chans_all_windows'] = {}

    sig_chans_per_subject['stim_sig_chans_timepermcluster'][sub] = stim_sig_chans
    sig_chans_per_subject['stim_sig_chans_0to0point5'][sub] = stim_sig_chans_0to0point5
    sig_chans_per_subject['stim_sig_chans_0point5to1'][sub] = stim_sig_chans_0point5to1
    sig_chans_per_subject['stim_sig_chans_0to1'][sub] = stim_sig_chans_0to1
    sig_chans_per_subject['stim_sig_chans_all_windows'][sub] = stim_sig_chans_all_windows

    num_sig_channels = {}
    num_sig_channels['stim_sig_chans_timepermcluster'] = len(sig_chans_per_subject['stim_sig_chans_timepermcluster'][sub])
    num_sig_channels['stim_sig_chans_0to0point5'] = len(sig_chans_per_subject['stim_sig_chans_0to0point5'][sub])
    num_sig_channels['stim_sig_chans_0point5to1'] = len(sig_chans_per_subject['stim_sig_chans_0point5to1'][sub])
    num_sig_channels['stim_sig_chans_0to1'] = len(sig_chans_per_subject['stim_sig_chans_0to1'][sub])
    num_sig_channels['stim_sig_chans_all_windows'] = len(sig_chans_per_subject['stim_sig_chans_all_windows'][sub])

    filt_channels_path = os.path.join(base_path, 'clean', f'sub-{sub}', 'ieeg', f'sub-{sub}_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv')
    with open(filt_channels_path, 'r') as file:
        filt_channels = file.readlines()  # Adjust this line if the data is stored in a different format (e.g., JSON)
    num_filt_channels = len(filt_channels) - 1

    # Update the counts
    counts['total_filt_channels'] += num_filt_channels
    counts['total_good_channels'] += num_good_channels

    counts['total_sig_channels_timepermcluster'] += num_sig_channels['stim_sig_chans_timepermcluster']
    counts['total_sig_channels_0to0point5'] += num_sig_channels['stim_sig_chans_0to0point5']
    counts['total_sig_channels_0point5to1'] += num_sig_channels['stim_sig_chans_0point5to1']
    counts['total_sig_channels_0to1'] += num_sig_channels['stim_sig_chans_0to1']
    counts['total_sig_channels_all_windows'] += num_sig_channels['stim_sig_chans_all_windows']


    # Print the counts for the current subject
    print(f"Subject {sub}: Filt Channels = {num_filt_channels}, Good Channels = {num_good_channels}, Sig channels time perm cluster = {num_sig_channels['stim_sig_chans_timepermcluster']}, Sig channels 0 to 0.5 = {num_sig_channels['stim_sig_chans_0to0point5']}, Sig channels 0.5 to 1 = {num_sig_channels['stim_sig_chans_0point5to1']}, Sig channels 0 to 1 = {num_sig_channels['stim_sig_chans_0to1']}, Sig Channels across windows = {num_sig_channels['stim_sig_chans_all_windows']}")

# Print total counts across all subjects
print("Total across all subjects:", counts)


Loaded significant channels for subject D0057
Loaded significant channels for subject D0057
Loaded significant channels for subject D0057
Loaded significant channels for subject D0057
Subject D0057: Filt Channels = 179, Good Channels = 176, Sig channels time perm cluster = 87, Sig channels 0 to 0.5 = 81, Sig channels 0.5 to 1 = 176, Sig channels 0 to 1 = 82, Sig Channels across windows = 176
Loaded significant channels for subject D0059
Loaded significant channels for subject D0059
Loaded significant channels for subject D0059
Loaded significant channels for subject D0059
Subject D0059: Filt Channels = 185, Good Channels = 174, Sig channels time perm cluster = 88, Sig channels 0 to 0.5 = 61, Sig channels 0.5 to 1 = 174, Sig channels 0 to 1 = 61, Sig Channels across windows = 174
Loaded significant channels for subject D0063
Loaded significant channels for subject D0063
Loaded significant channels for subject D0063
Loaded significant channels for subject D0063
Subject D0063: Filt Channe

In [25]:
sig_chans_per_subject['stim_sig_chans_0to0point5']

{'D0103': ['LTAS1',
  'LTMS7',
  'LTPS13',
  'LTPS16',
  'LFAM8',
  'LFAM11',
  'LFAM12',
  'LFAM13',
  'LFAM14',
  'LAI1',
  'LAI4',
  'LAI6',
  'LAI7',
  'LAI8',
  'LPI2',
  'LPI3',
  'LPI5',
  'LPI6',
  'LPI7',
  'LPI15',
  'LPI16',
  'LPI17',
  'LTP2',
  'LTP3',
  'LTP4',
  'LTPM6',
  'LTPM9',
  'LTPM10',
  'LPPI7',
  'LPPI8',
  'LPPI9',
  'LPPI18',
  'LTLI8',
  'LTLI9',
  'LTPI2',
  'LTPI5',
  'LTPI7',
  'LFO1',
  'LFO2',
  'LFO3',
  'LFO4',
  'LFO5',
  'LFO6',
  'LFO7',
  'LFO8',
  'LFO9',
  'LFO10',
  'LFO12',
  'LFO14',
  'LFO15',
  'LFAI1',
  'LFAI2',
  'LFAI3',
  'LFAI4']}